## Twitter Data Predictions

In [ ]:
# Import relevant packages
import os
import tweepy
import pandas as pd
from matplotlib.dates import DateFormatter

First, I will have to gain access to the Twitter API, using my keys and secrets.

In [ ]:
# define your keys
consumer_key = '2YCaHB1rnU7I7U8BuDJVqPGP2'
consumer_secret = 'UJR0oFVc6JzaoWC6J2K2n3cMEfdAZS6nhJtHTGeHBnehrFVPZw'
access_token = '1280193789756309511-b6F7ZCckvK3crRh7EzhfKk0sIJBYXQ'
access_token_secret = '1I3YLccFnFoGzP0ekSWGLgXdUVBFHyVhnmvOs2ZQWX1XX'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

Then, from the Tweepy API, I'm interested to find out what is the top 10 trending topics for Canada currently:

In [ ]:
from tweepy import API
WOEID = 2972 # WOEID for Canada
# Returns the top 50 trending topics worldwide
trends1 = api.trends_place(id = WOEID)
# printing and listing the information
print("The top 10 trends for the location are :") 
topics = []
for value in trends1: 
    for trend in value['trends'][0:10]:
        topics.append(trend['name'].replace('#',''))
        print(trend['name'].replace('#',''))

### Retrieve Topic-Specific Tweets

First, I will have to specify the period of time I want to retrieve my tweets from, in this case, I have specified the API to return tweets from the last 7 days:

In [ ]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = (date.today() + timedelta(days=-6))
end_date = (date.today() + timedelta(days=1))
date_list = []
for single_date in daterange(start_date, end_date):
    date_list.append(single_date)

Then, using the `tweepy.Cursor` method, I was able to pull the most 1000 recent tweets from the last 7 days, with specific conditions like topics (text_query), location (coordinates), language and number of tweets returned per day:

In [ ]:
tweets_list_date = list()
tweets_list_text = list()

for date_ in date_list:
    text_query = "Raptors"
    coordinates = '43.651070,-79.347015,50mi'
    language = 'en'
    result_type = 'recent'
    since_date = date_
    until_date = (date_ + timedelta(days=1))
    max_tweets = 1000
    
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search,
                           q = text_query,
                           geocode = coordinates,
                           lang=language,
                           result_type = result_type,
                           since = since_date,
                           until = until_date,
                           count = 100).items(max_tweets)
    
    # List comprehension pulling chosen tweet information from tweets iterable object
    for tweet in tweets:
        tweets_list_date.append(tweet.created_at)
        tweets_list_text.append(tweet.text)

In [ ]:
# Creation of dataframe from tweets_list
tweets_df = pd.DataFrame({'date' : tweets_list_date,'text' : tweets_list_text},columns=['date','text'])
# Add an independent column date
date = tweets_df['date']
date = pd.to_datetime(date).dt.date
tweets_df

### Feature Engineering

The feature engineering process is more or less similar to the process for the train dataset, including:
- new date-time columns
- hashtags and their counts
- mentions and their counts
- url and their counts

The biggest difference, would be not removing the emojis from the text column (yet) for the emoji visualization later in the notebook.

In [ ]:
# Hours
tweets_df['hour'] = [dt.hour for dt in tweets_df['date'].astype(object)]
# Days
tweets_df['day'] = [dt.day for dt in tweets_df['date'].astype(object)]
# Month
tweets_df['month'] = [dt.month for dt in tweets_df['date'].astype(object)]
# Weekday
tweets_df['dayofweek'] = [dt.dayofweek for dt in tweets_df['date'].astype(object)]
# Delete date column
tweets_df = tweets_df.drop(['date'],axis=1)

In [ ]:
# Retrieve the hashtags and add the column to the dataset
hashtags = []
for tweet in tweets_df['text']:
    hashtags.append([i  for i in tweet.split() if i.startswith("#") ])
tweets_df['hashtags'] = hashtags
# Find number of hashtags in each tweet
hashtag_counts = []
for hashtag in hashtags:
    hashtag_counts.append(len(hashtag))
tweets_df['hashtag_counts'] = hashtag_counts

In [ ]:
# Retrieve the user names and add the column to the dataset
users = []
for tweet in tweets_df['text']:
    users.append([i for i in tweet.split() if i.startswith("@") ])
tweets_df['users'] = users
# Find number of tagged users in each tweet
user_counts = []
for user in users:
    user_counts.append(len(user))
tweets_df['user_counts'] = user_counts
# Drop users column
tweets_df = tweets_df.drop(['users'],axis=1)

In [ ]:
# Retrieve the URLs from the tweets
from urlextract import URLExtract
extractor = URLExtract()
urls = []
for i in range(len(tweets_df)):
    urls.append(extractor.find_urls(tweets_df['text'][i]))
tweets_df['urls'] = urls
# Find number of urls in each tweet
url_counts = []
for url in tweets_df['urls']:
    url_counts.append(len(url))
tweets_df['url_counts'] = url_counts
# Drop urls column
tweets_df = tweets_df.drop(['urls'],axis=1)

In [ ]:
# Remove excessive information from text Column
import re
# Creating a function called clean, that removes all hyperlink, hashtags and mentions
def clean(x):
    x = re.sub(r"^RT[\s]+", "", x)
    x = re.sub(r"https?:\/\/.*[\r\n]*", "", x)
    #x = re.sub('[^ ]+\.[^ ]+','',x)
    x = re.sub(r"#","", x)
    x = re.sub(r"@[A-Za-z0–9]+","", x)

    return x  
# Apply the clean function to text column
tweets_df['text'] = tweets_df['text'].apply(clean)

In [ ]:
tweets_df

### Vectorization

The vectorization process is the same as well, except we are specifying the vocabulary of the vectorizer from the `reduced_feature.pkl` saved from the earlier notebook.

In [ ]:
# Import the relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

# Load features from training dataset
transformer = TfidfTransformer()
loaded_features = pickle.load(open("pickle/reduced_feature.pkl", "rb"))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Vectorize the text column
X_text = tweets_df['text'].astype(str)
tfidfconverter = TfidfVectorizer(max_features=10000, 
                                 min_df=0.0001, max_df=0.5, 
                                 stop_words=stopwords.words('english'),
                                 token_pattern=r'\b[^\d\W]+\b',
                                 strip_accents = "ascii",
                                 vocabulary = loaded_features)

In [ ]:
# Convert the features in test set to train set
X_text = transformer.fit_transform(tfidfconverter.fit_transform(X_text))

In [ ]:
X_text = pd.DataFrame(columns=tfidfconverter.get_feature_names(),data=X_text.toarray())
# Retrieve the numerical columns
X_num = tweets_df.drop(['text','hashtags'],axis=1)
# Concatenate the test dataset
X_sample = pd.concat([X_num,X_text],axis=1).astype('int64')

### Prediction & Visualizations

After preprocessing and vectorization, we can pass our new dataset through the trained models, in this case, I have selected the logistic model trained with reduced dataset, as so far, comparing to the model trained with incremental learning, it appears to perform better.

#### Sentiment Analysis

This is the sentimental analysis for the topic of Raptors over the last 7 days, where the sentiment score falls on the range of -1 to 1, with -1 being the most negative and 1 being the most positive.

In [ ]:
# load the model from disk
filename = 'pickle/svc_reduced_model.sav'
svc_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_sample = svc_model.predict(X_sample)

In [ ]:
y_prediction = pd.DataFrame(y_sample,columns = ["prediction"])
y_prediction = pd.concat([date,y_prediction],axis=1)

In [ ]:
y_prediction_table = y_prediction.groupby('date').mean()

In [ ]:
y_prediction_table.to_csv('howistwitterfeeling/src/streamlit/covid_demo.csv')
tweets_df.to_csv('howistwitterfeeling/src/streamlit/covid_tweets.csv')

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(12, 6))
# Add x-axis and y-axis
ax.plot(y_prediction_table.index,
           y_prediction_table.prediction,
       color='purple')
# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="",)

# Set limit for y scale
ax.set_ylim(y_prediction_table.prediction.min()-0.01,y_prediction_table.prediction.max()+0.03)

# Define the date format
date_form = DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)
plt.show()

#### Top 10 Emojis Associated with Topic

In [ ]:
import emoji
import regex

# Create a function, where emojis found in the texts associated with the topic are returned
def split_count(text):

    '''
    Return the emojis found in the twitter texts associated with the topic
    Input: text column from dataset
    Output: A list of emojis found in each rows
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

# Return the emojis found in the twitter texts
emoji_rows = tweets_df['text'].apply(split_count)

# Return a flattened list of emojis from the topic
emoji_list = []
for sublist in emoji_rows:
    for item in sublist:
        emoji_list.append(item)
        
emoji_count = [[x,emoji_list.count(x)] for x in set(emoji_list)]

In [ ]:
# Convert the list into a dataframe
emoji_df = pd.DataFrame(emoji_count,columns=['Emoji','Count']).sort_values('Count',ascending=False)

In [ ]:
# print top 10 emojis
emoji_df.head(10)

#### Hashtag Wordcloud

In [ ]:
# Return a flattened list of emojis from the topic
hashtag_list = []
for sublist in tweets_df['hashtags']:
    for item in sublist:
        hashtag_list.append(item)
hashtag_list

hashtag_str = ' '.join(hashtag for hashtag in hashtag_list)

In [ ]:
import matplotlib.pyplot as pPlot
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(collocations=False).generate(hashtag_str)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()